In [1]:
from bs4 import BeautifulSoup
import pandas as pd
import requests
import urllib.request
import time

In [28]:
# create lists to store the scraped data
authors=[]
dates=[]
statements=[]
sources=[]
targets=[]

 HTML parsing is basically: taking in HTML code and extracting relevant information like the title of the page, paragraphs in the page, headings in the page, links, bold text etc.

In [32]:
# create a function to scrap the website
def scrap_website(page):
    page_num=str(page)
    URL="https://www.politifact.com/factchecks/list/?page="+page_num
    webpage=requests.get(URL)
    soup=BeautifulSoup(webpage.text,"html.parser")
    #Get the location of the information
    statement_footer=soup.find_all('footer',attrs={"class":"m-statement__footer"})
    statement_quote=soup.find_all("div",attrs={'class':'m-statement__quote'})
    statement_meta=soup.find_all("div",attrs={"class":"m-statement__meta"})
    target=soup.find_all("div",attrs={'class':'m-statement__meter'})
    
    # Loop through statement_footer
    for i in statement_footer:
        link1=i.text.strip()
        name_and_date=link1.split()
        first_name=name_and_date[1]
        last_name=name_and_date[2]
        full_name=first_name + " " + last_name
        month=name_and_date[4]
        day=name_and_date[5]
        year=name_and_date[6]
        date=month + " " + day + " " + year
        dates.append(date)
        authors.append(full_name)
    
    # loop through statement_quote
    for i in statement_quote:
        link2=i.find_all("a")
        statement_text=link2[0].text.strip()
        statements.append(statement_text)
        
    # loop throug the meta
    for i in statement_meta:
        link3=i.find_all('a')
        source_text=link3[0].text.strip()
        sources.append(source_text)
        
    # Loop through the target
    for i in target:
        link4=i.find('div',attrs={'class':'c-image'}).find('img').get('alt')
        targets.append(link4)
        

In [33]:
# loop through 'n-1' webpages to scrape the data
n=2
for i in range(1,n):
    scrap_website(i)
    

In [34]:
# create the dataframe
data=pd.DataFrame(columns=['author','statement','source','date','target'])
data['author']=authors
data['statement']=statements
data['source']=sources
data['date']=dates
data['target']=targets
data

,author,statement,source,date,target
0,Samantha Putterman,The New York Times published an old stock phot...,Facebook posts,"June 3, 2021",mostly-true
1,Michael Majchrowicz,"""The CDC lowered the PCR threshold while no lo...",Tweets,"June 3, 2021",false
2,Jon Greenberg,"Emails show ""Fauci's fellow scientist could te...",Facebook posts,"June 2, 2021",false
3,Louis Jacobson,"""Two members of the Senate,"" Joe Manchin and K...",Joe Biden,"June 2, 2021",barely-true
4,Tom Kertscher,"An email from Dr. Anthony Fauci shows ""everyon...",Instagram posts,"June 2, 2021",false
5,Louis Jacobson,"“The filibuster, if it does not get reformed, ...",Barack Obama,"June 2, 2021",half-true
6,Ciara O'Rourke,“U.S. military arrests Dr. Deborah Birx.”,Viral image,"June 2, 2021",false
7,Madeline Heim,"“Since 1978, CEO compensation rose over 1,000%...",Mandela Barnes,"June 2, 2021",true
8,Monique Curet,“So the unvaxxed have to wear mask (sic) to pr...,Facebook posts,"June 2, 2021",false
9,Samantha Putterman,“Not one word” from Joe Biden and Kamala Harri...,Facebook posts,"June 1, 2021",false


In [35]:
# Create a funtion to get a binary label true or false from the target
def getBinaryTarget(text):
    if text=='true':
        return "Real"
    else:
        return "Fake"
    

In [36]:
# creating new columns
data["BinaryTarget"]=data["target"].apply(getBinaryTarget)

In [37]:
data

,author,statement,source,date,target,BinaryTarget
0,Samantha Putterman,The New York Times published an old stock phot...,Facebook posts,"June 3, 2021",mostly-true,Fake
1,Michael Majchrowicz,"""The CDC lowered the PCR threshold while no lo...",Tweets,"June 3, 2021",false,Fake
2,Jon Greenberg,"Emails show ""Fauci's fellow scientist could te...",Facebook posts,"June 2, 2021",false,Fake
3,Louis Jacobson,"""Two members of the Senate,"" Joe Manchin and K...",Joe Biden,"June 2, 2021",barely-true,Fake
4,Tom Kertscher,"An email from Dr. Anthony Fauci shows ""everyon...",Instagram posts,"June 2, 2021",false,Fake
5,Louis Jacobson,"“The filibuster, if it does not get reformed, ...",Barack Obama,"June 2, 2021",half-true,Fake
6,Ciara O'Rourke,“U.S. military arrests Dr. Deborah Birx.”,Viral image,"June 2, 2021",false,Fake
7,Madeline Heim,"“Since 1978, CEO compensation rose over 1,000%...",Mandela Barnes,"June 2, 2021",true,Real
8,Monique Curet,“So the unvaxxed have to wear mask (sic) to pr...,Facebook posts,"June 2, 2021",false,Fake
9,Samantha Putterman,“Not one word” from Joe Biden and Kamala Harri...,Facebook posts,"June 1, 2021",false,Fake


# Flipcart Data

In [164]:
Mobile=[]
Price=[]
Discount=[]
Tmp=[]
Specs=[]
def scrap(url):
    webpage=requests.get(url)
    soup=BeautifulSoup(webpage.text,"html.parser")
    mobile=soup.find_all("div",attrs={'class':'_4rR01T'})
    price=soup.find_all("div",attrs={'class':'_30jeq3 _1_WHN1'})
    specs=soup.find_all("li",attrs={'class':'rgWa7D'})
    discount=soup.find_all("div",attrs={'class':'_3Ay6Sb'})
    tmp=soup.find_all("div",attrs={'class':'fMghEO'})
    
    
    for i in mobile:
        Mobile.append(i.text)
        
    for i in price:
        Price.append(i.text)
        
    for i in discount:
        Discount.append(i.text)
        
    for i in specs:
        Specs.append(i.text)
    
    for i in tmp:
        Tmp.append(i.text)

In [165]:
scrap("https://www.flipkart.com/mobiles/~cs-tcwxykrrxv/pr?sid=tyy%2C4io&collection-tab-name=realme+X7+Max+Series&sort=price_asc&param=345&otracker=clp_bannerads_1_5.bannerAdCard.BANNERADS_Realme%2BX7%2BMax%2B_mobile-phones-store_4PMT0WB0L0EP")

In [180]:
RAM_ROM=[]
Display=[]
Camera=[]
Battery=[]
Processor=[]
Warranty=[]
for i in range(0,len(Specs),6):
    RAM_ROM.append(Specs[i])
    Display.append(Specs[i+1])
    Camera.append(Specs[i+2])
    Battery.append(Specs[i+3])
    Processor.append(Specs[i+4])
    Warranty.append(Specs[i+5])

In [187]:
rows=[Mobile,RAM_ROM,Display,Camera,Battery,Processor,Warranty,Discount,Price]
df=pd.DataFrame(rows).T
columns=["Mobile","RAM_ROM","Display","Camera","Battery","Processor","Warranty","Discount","Price"]
df.columns=columns
df

,Mobile,RAM_ROM,Display,Camera,Battery,Processor,Warranty,Discount,Price
0,"realme X7 Max (Mercury Silver, 128 GB)",8 GB RAM | 128 GB ROM,16.33 cm (6.43 inch) Full HD+ Display,64MP + 8MP + 2MP | 16MP Front Camera,4500 mAh Battery,MediaTek Dimensity 1200 Processor,1 Year Warranty for Mobile and 6 Months for Ac...,10% off,"₹26,999"
1,"realme X7 Max (Asteroid Black, 128 GB)",8 GB RAM | 128 GB ROM,16.33 cm (6.43 inch) Full HD+ Display,64MP + 8MP + 2MP | 16MP Front Camera,4500 mAh Battery,MediaTek Dimensity 1200 Processor,1 Year Warranty for Mobile and 6 Months for Ac...,10% off,"₹26,999"
2,"realme X7 Max (Asteroid Black, 256 GB)",12 GB RAM | 256 GB ROM,16.33 cm (6.43 inch) Full HD+ Display,64MP + 8MP + 2MP | 16MP Front Camera,4500 mAh Battery,MediaTek Dimensity 1200 Processor,1 Year Warranty for Mobile and 6 Months for Ac...,9% off,"₹29,999"
3,"realme X7 Max (Mercury Silver, 256 GB)",12 GB RAM | 256 GB ROM,16.33 cm (6.43 inch) Full HD+ Display,64MP + 8MP + 2MP | 16MP Front Camera,4500 mAh Battery,MediaTek Dimensity 1200 Processor,1 Year Warranty for Mobile and 6 Months for Ac...,9% off,"₹29,999"
4,"realme X7 Max (Milky Way, 128 GB)",8 GB RAM | 128 GB ROM,16.33 cm (6.43 inch) Full HD+ Display,64MP + 8MP + 2MP | 16MP Front Camera,4500 mAh Battery,MediaTek Dimensity 1200 Processor,1 Year Warranty for Mobile and 6 Months for Ac...,10% off,"₹26,999"
5,"realme X7 Max (Milky Way, 256 GB)",12 GB RAM | 256 GB ROM,16.33 cm (6.43 inch) Full HD+ Display,64MP + 8MP + 2MP | 16MP Front Camera,4500 mAh Battery,MediaTek Dimensity 1200 Processor,1 Year Warranty for Mobile and 6 Months for Ac...,9% off,"₹29,999"


# Flipcart Footwear Dataset

In [195]:
Company=[]
Usage=[]
Discount=[]
Price=[]
def scrap_pages(starting_page_number,ending_page_number):
    for p in range(starting_page_number,ending_page_number):
        p=str(p)
        url="https://www.flipkart.com/mens-footwear/pr?sid=osp%2Ccil&p%5B%5D=facets.fulfilled_by%255B%255D%3DFlipkart%2BAssured&sort=popularity&p%5B%5D=facets.brand%255B%255D%3DADIDAS&p%5B%5D=facets.brand%255B%255D%3DADIDAS%2BORIGINALS&p%5B%5D=facets.brand%255B%255D%3DADIDAS%2BNEO&p%5B%5D=facets.brand%255B%255D%3DNike&p%5B%5D=facets.brand%255B%255D%3DAsics&p%5B%5D=facets.brand%255B%255D%3DASICS%2BTIGER&p%5B%5D=facets.brand%255B%255D%3DAsics%2BTiger&p%5B%5D=facets.brand%255B%255D%3DAsics%2BTIGER&p%5B%5D=facets.brand%255B%255D%3DPuma&p%5B%5D=facets.brand%255B%255D%3DREEBOK&p%5B%5D=facets.brand%255B%255D%3DREEBOK%2BCLASSICS&p%5B%5D=facets.brand%255B%255D%3DSkechers&p%5B%5D=facets.brand%255B%255D%3DLotto&p%5B%5D=facets.brand%255B%255D%3DYonex&p%5B%5D=facets.brand%255B%255D%3DWildcraft&p%5B%5D=facets.brand%255B%255D%3DCampus&p%5B%5D=facets.brand%255B%255D%3DUnder%2BArmour&p%5B%5D=facets.brand%255B%255D%3DASICS%2BTiger&p%5B%5D=facets.brand%255B%255D%3DUNDER%2BARMOUR&p%5B%5D=facets.brand%255B%255D%3DFila&p%5B%5D=facets.brand%255B%255D%3DADIDAS%2BOriginals&p%5B%5D=facets.price_range.from%3D300&p%5B%5D=facets.price_range.to%3DMax&hpid=6U94QwEnuAvj8P8OMld0-ap7_Hsxr70nj65vMAAFKlc%3D&fm=neon%2Fmerchandising&iid=M_f60a9265-9556-41d1-a218-23d1a8ce1e09_14.VPVKQ54UBPX8&ppt=hp&ppn=homepage&ssid=3ppaxbf8cg0000001622751358668&otracker=hp_omu_Men%2527s%2BFootwear%2B_1_12.dealCard.OMU_VPVKQ54UBPX8_8&otracker1=hp_omu_WHITELISTED_neon%2Fmerchandising_Men%2527s%2BFootwear%2B_NA_dealCard_cc_1_NA_view-all_8&cid=VPVKQ54UBPX8&page="+p
        webpage=requests.get(url)
        soup=BeautifulSoup(webpage.text,"html.parser")
        company=soup.find_all('div',attrs={'class':'_2WkVRV'})
        usage=soup.find_all('a',attrs={'class':'IRpwTa'})
        price=soup.find_all('div',attrs={'class':'_30jeq3'})
        discount=soup.find_all('div',attrs={'class':'_3Ay6Sb'})
        
        for i in company:
            Company.append(i.text)
            
        for i in usage:
            Usage.append(i.text)
        
        for i in discount:
            Discount.append(i.text)
            
        for i in price:
            Price.append(i.text)
            
            
        
        

In [203]:
scrap_pages(0,10)

In [204]:
rows=[Company,Usage,Discount,Price]
columns=['Company','Usage','Discount','Price']
footwear_data=pd.DataFrame(rows).T
footwear_data.columns=columns
footwear_data

,Company,Usage,Discount,Price
0,PUMA,Zod Runner V3 IDP Running Shoes For Men,62% off,"₹1,499"
1,CAMPUS,OXYFIT Casuals For Men,16% off,₹671
2,CAMPUS,CRYSTA Running Shoes For Men,21% off,₹825
3,REEBOK,Stride Runner Running Shoes For Men,30% off,"₹1,314"
4,CAMPUS,ROYCE-2 Running Shoes For Men,10% off,₹939
...,...,...,...,...
515,PUMA,Lazy Knit Slip On IDP Sneakers For Men,None,"₹1,379"
516,PUMA,Cape IDP Sneakers For Men,None,"₹2,010"
517,REEBOK,Men Blue Sports Sandal,None,₹997
518,PUMA,NRGY Asteroid Running Shoes For Men,None,"₹1,919"
